In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

!pip install --upgrade transformers==4.53.3 datasets[audio] librosa tqdm torchcodec accelerate jiwer

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.9/832.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 

In [2]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    WhisperProcessor,
    WhisperForConditionalGeneration,
)


2025-10-27 16:53:32.392795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761584012.590931      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761584012.647371      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import os
from datasets import load_from_disk, Audio, DatasetDict, disable_caching, enable_caching

# 🔹 1. Load dataset
vimd = load_from_disk("/kaggle/input/whisperdataset/ViSpeech_with_text")

# ⭐ FIX: TẮT Caching trước khi thực hiện các biến đổi
disable_caching()

# Chuẩn hoá audio về 16kHz (cast_column cũng tạo cache)
vimd = DatasetDict({
    split: vimd[split].cast_column("audio", Audio(sampling_rate=16000))
    for split in vimd
})

def add_condition_tags(example):
    dialect_tag = f"[{example['dialect']}]"
    gender_tag = f"[{example['gender']}]"
    example["text"] = f"{dialect_tag} {gender_tag} {example['text']}"
    return example

# Thao tác map (bây giờ không cần load_from_cache_file=False nữa)
vimd = vimd.map(add_condition_tags)

# Tùy chọn: Bật lại caching nếu cần cho các thao tác sau
enable_caching() 


print(vimd)
print(vimd["train"][0])

Map:   0%|          | 0/11752 [00:00<?, ? examples/s]

Map:   0%|          | 0/2982 [00:00<?, ? examples/s]

Map:   0%|          | 0/4978 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'gender', 'dialect', 'text', 'log_info'],
        num_rows: 11752
    })
    validation: Dataset({
        features: ['audio', 'gender', 'dialect', 'text', 'log_info'],
        num_rows: 2982
    })
    test: Dataset({
        features: ['audio', 'gender', 'dialect', 'text', 'log_info'],
        num_rows: 4978
    })
})
{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x787ef0e787d0>, 'gender': 'female', 'dialect': 'northern dialect', 'text': '[northern dialect] [female] ngày có mưa rào và dông vải nơi do tây đến tây nam cấp hai cấp ba nhân độ từ hai mươi ba đến ba mươi mốt độ.', 'log_info': '✅ Done in 2.72s'}


In [4]:
processor = WhisperProcessor.from_pretrained("vinai/PhoWhisper-base", language="vi", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("vinai/PhoWhisper-base")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="vi", task="transcribe")
model.config.suppress_tokens = []
new_tokens = ["[northern dialect]", "[central dialect]", "[southern dialect]", "[male]", "[female]"]
processor.tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(processor.tokenizer))


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Embedding(50369, 512, padding_idx=50257)

In [5]:
def prepare_batch(batch):
    audio = batch["audio"]
    inputs = processor.feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    )
    batch["input_features"] = inputs["input_features"][0]

    batch["labels"] = processor.tokenizer(
    batch["text"], truncation=True, max_length=448
).input_ids
    return batch

#dataset = vimd.map(prepare_batch, remove_columns=["audio", "text"], load_from_cache_file=False)

In [6]:
dataset = vimd.map(prepare_batch, remove_columns=["audio"], load_from_cache_file=False)

Map:   0%|          | 0/11752 [00:00<?, ? examples/s]

Map:   0%|          | 0/2982 [00:00<?, ? examples/s]

Map:   0%|          | 0/4978 [00:00<?, ? examples/s]

In [7]:
dataset.shape

{'train': (11752, 6), 'validation': (2982, 6), 'test': (4978, 6)}

In [8]:
# Lưu toàn bộ DatasetDict sau khi xử lý
dataset.save_to_disk("/kaggle/working/vimd_prepared")
print("✅ Dataset đã được lưu tại /kaggle/working/vimd_prepared")


Saving the dataset (0/23 shards):   0%|          | 0/11752 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/2982 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/4978 [00:00<?, ? examples/s]

✅ Dataset đã được lưu tại /kaggle/working/vimd_prepared


In [9]:
# # Lấy 1 phần nhỏ của mỗi split để test
# small_dataset = {
#     split: dataset_text[split].select(range(min(100, len(dataset_text[split]))))  # chọn 100 mẫu đầu
#     for split in dataset_text
# }

# for k, v in small_dataset.items():
#     print(f"{k}: {len(v)} samples")


In [10]:
# from dataclasses import dataclass
# from typing import Any, Dict, List, Union
# import torch

# @dataclass
# class DataCollatorSpeechSeq2SeqWithPadding:
#     processor: Any
#     pad_to_multiple_of: Union[int, None] = None

#     def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
#         # Tách phần input (audio) và label (text)
#         input_features = [{"input_features": f["input_features"]} for f in features]
#         label_features = [{"input_ids": f["labels"]} for f in features]

#         # Padding audio feature
#         batch = self.processor.feature_extractor.pad(
#             input_features,
#             padding=True,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors="pt"
#         )

#         # Padding text label
#         labels_batch = self.processor.tokenizer.pad(
#             label_features,
#             padding=True,
#             return_tensors="pt"
#         )

#         # Thay padding id trong labels thành -100 (để bỏ qua trong loss)
#         labels = labels_batch["input_ids"].masked_fill(
#             labels_batch.attention_mask.ne(1), -100
#         )
#         batch["labels"] = labels

#         return batch


In [11]:
# import jiwer, numpy as np

# def compute_metrics(pred):
#     # pred.predictions là list các chuỗi token IDs do model.generate() trả về
#     pred_ids = pred.predictions

#     # decode predicted text
#     pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)

#     # decode reference text (labels)
#     label_ids = pred.label_ids
#     label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
#     label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

#     wer = jiwer.wer(label_str, pred_str)
#     return {"wer": wer}



In [12]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="pho-whisper-vimd-conditioned",
#     num_train_epochs=1,          # chỉ chạy 1 vòng qua dataset
#     per_device_train_batch_size=8,
#     gradient_accumulation_steps=1,
#     learning_rate=1e-5,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     fp16=False,
#     predict_with_generate=False,
#     report_to=[],
#     remove_unused_columns=False,
# )

In [13]:
# data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor, pad_to_multiple_of=8)

# # training_args = Seq2SeqTrainingArguments(
# #     output_dir="pho-whisper-vimd-conditioned",
# #     per_device_train_batch_size=8,
# #     gradient_accumulation_steps=2,
# #     learning_rate=1e-5,
# #     warmup_steps=500,
# #     num_train_epochs=3,
# #     eval_strategy="steps",
# #     eval_steps=500,
# #     save_steps=500,
# #     save_total_limit=2,
# #     fp16=True,
# #     predict_with_generate=True,
# #     report_to=["tensorboard"],
# #     remove_unused_columns=False,
# #     load_best_model_at_end=True,
# #     metric_for_best_model="wer",
# #     greater_is_better=False,
# # )



In [14]:
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_dataset["train"],
#     eval_dataset=small_dataset["validation"],
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )
# trainer.train()

In [15]:
# results = {}
# for d in ["northern dialect", "central dialect", "southern dialect"]:
#     subset = dataset["test"].filter(lambda x: x["dialect"] == d)
#     out = trainer.evaluate(subset)
#     results[d] = out["eval_wer"]
# print(results)
